# Feature creation and translation to wide df

In [16]:
######################## LOAD IN FILES #############################
import os
import pandas as pd
import sys
import importlib

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Define data directory
brighten_dir = os.path.join(project_root, 'BRIGHTEN_data')

# Add project root to sys.path for script usage
sys.path.append(project_root)

# Import and reload (optional) custom scripts
# Import and reload custom scripts
from scripts import preprocessing as pre
from scripts import visualization as vis
from scripts import feature_selection as fs
from scripts import clustering as cl
from scripts import variables as var
importlib.reload(pre)
importlib.reload(vis)
importlib.reload(fs)
importlib.reload(cl)
importlib.reload(var)

################ DEFINE column variables from data ###################
from scripts.variables import id_columns, daily_cols_v1, daily_v2_common 
from scripts.variables import phq2_cols, phq9_cols, weekly_cols, passive_cols, survey_cols
from scripts.variables import df_names, df_names_with_mis

############ Import compiled raw data ############
df_alldays_int70 = pd.read_csv(os.path.join(brighten_dir, 'df_alldays_int70.csv'))
week_df_int70 = pd.read_csv(os.path.join(brighten_dir, 'week_df_int70.csv'))




/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_74668/514711111.py:34: DtypeWarning: Columns (14,23,48,106,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_alldays_int70 = pd.read_csv(os.path.join(brighten_dir, 'df_alldays_int70.csv'))
/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_74668/514711111.py:35: DtypeWarning: Columns (24,107,108,109) have mixed types. Specify dtype option on import or set low_memory=False.
  week_df_int70 = pd.read_csv(os.path.join(brighten_dir, 'week_df_int70.csv'))


In [24]:

for name in df_names_with_mis:
    if os.path.exists(os.path.join(brighten_dir, f'{name}_pca.csv')):
        df = pd.read_csv(os.path.join(brighten_dir, f'{name}_pca.csv'))
    else:
        df = pd.read_csv(os.path.join(brighten_dir, f'{name}_scaled.csv'))
    if 'day' in name:
        if 'phq2_bin' not in df.columns.to_list():
            df_bin = df.merge(df_alldays_int70[id_columns + ['phq2_bin']], on=id_columns, how='left')
        else:
            df_bin = df
    elif 'week' in name:
        merge_cols = [col for col in ['phq2_bin', 'phq9_bin', 'phq9_cat'] if col not in df.columns.to_list()]
        if len(merge_cols) > 0:
            df_bin = df.merge(df_alldays_int70[id_columns + merge_cols], on=id_columns, how='left')
        else:
            df_bin = bin

    df_bin.to_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))





/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_74668/336776481.py:14: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df_bin = df.merge(df_alldays_int70[id_columns + merge_cols], on=id_columns, how='left')
/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_74668/336776481.py:14: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df_bin = df.merge(df_alldays_int70[id_columns + merge_cols], on=id_columns, how='left')


In [ ]:
### making features -- averaging of time data

for name in df_names_with_mis:
    df_bin = pd.read_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))
    wide_df_bin = pre.make_wide_df(df_bin, id_columns)
    wide_df_bin.to_csv(os.path.join(brighten_dir, f'wide_{name}_bin.csv'))




In [29]:
for name in df_names_with_mis:
    wide_df_bin = pd.read_csv(os.path.join(brighten_dir, f'wide_{name}_bin.csv'))
    print(name, wide_df_bin.columns.to_list())

v1_day ['Unnamed: 0', 'participant_id', 'num_id', 'mobility_radius_avg_w1', 'mobility_radius_avg_wu2', 'mobility_radius_slope_wu2', 'mobility_radius_avg_wu4', 'mobility_radius_slope_wu4', 'mobility_radius_avg_w4', 'pc_communication_avg_w1', 'pc_communication_avg_wu2', 'pc_communication_slope_wu2', 'pc_communication_avg_wu4', 'pc_communication_slope_wu4', 'pc_communication_avg_w4', 'phq2_bin_avg_w1', 'phq2_bin_avg_wu2', 'phq2_bin_slope_wu2', 'phq2_bin_avg_wu4', 'phq2_bin_slope_wu4', 'phq2_bin_avg_w4', 'missed_interactions_avg_w1', 'missed_interactions_avg_wu2', 'missed_interactions_slope_wu2', 'missed_interactions_avg_wu4', 'missed_interactions_slope_wu4', 'missed_interactions_avg_w4', 'aggregate_communication_avg_w1', 'aggregate_communication_avg_wu2', 'aggregate_communication_slope_wu2', 'aggregate_communication_avg_wu4', 'aggregate_communication_slope_wu4', 'aggregate_communication_avg_w4', 'call_count_avg_w1', 'call_count_avg_wu2', 'call_count_slope_wu2', 'call_count_avg_wu4', 'call

In [30]:

# Add binary
for name in df_names_with_mis:
    df_pca_bin = pd.read_csv(os.path.join(brighten_dir, f'wide_{name}_bin.csv'))
    if 'day' in name:
        df_pca_bin_day = pre.round_vars_phq2(df_pca_bin)
        df_pca_bin_day = df_pca_bin_day.dropna()
        df_pca_bin_day.to_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))
    if 'week' in name:
        df_pca_bin_week = pre.round_vars_phq9(df_pca_bin)
        df_pca_bin_week.to_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))


In [ ]:
demo_clusters = pd.read_csv(os.path.join(brighten_dir, 'demo_clusters.csv'))
# Merge in demographic clusters
for name in df_names_with_mis:
    df_bin = pd.read_csv(os.path.join(brighten_dir, f'{name}_bin.csv'))
    df_clusters = df_bin.merge(demo_clusters[['num_id', 'Cluster']], on=['num_id'], how='left')
    df_clusters_merged = df_clusters.merge(df_alldays_int70[['num_id']], on=['num_id'], how='left')
    df_clusters = df_clusters_merged.drop_duplicates().dropna()
    df_clusters = df_clusters.drop(columns=[col for col in df.columns if "Unnamed" in col or "0" in col])
    print(df_clusters.columns.to_list())
    df_clusters.to_csv(os.path.join(brighten_dir, f'wide_{name}_clusters.csv'))


['Unnamed: 0.1', 'Unnamed: 0', 'participant_id', 'num_id', 'mobility_radius_avg_w1', 'mobility_radius_avg_wu2', 'mobility_radius_slope_wu2', 'mobility_radius_avg_wu4', 'mobility_radius_slope_wu4', 'mobility_radius_avg_w4', 'pc_communication_avg_w1', 'pc_communication_avg_wu2', 'pc_communication_slope_wu2', 'pc_communication_avg_wu4', 'pc_communication_slope_wu4', 'pc_communication_avg_w4', 'phq2_bin_avg_w1', 'phq2_bin_avg_wu2', 'phq2_bin_slope_wu2', 'phq2_bin_avg_wu4', 'phq2_bin_slope_wu4', 'phq2_bin_avg_w4', 'missed_interactions_avg_w1', 'missed_interactions_avg_wu2', 'missed_interactions_slope_wu2', 'missed_interactions_avg_wu4', 'missed_interactions_slope_wu4', 'missed_interactions_avg_w4', 'aggregate_communication_avg_w1', 'aggregate_communication_avg_wu2', 'aggregate_communication_slope_wu2', 'aggregate_communication_avg_wu4', 'aggregate_communication_slope_wu4', 'aggregate_communication_avg_w4', 'call_count_avg_w1', 'call_count_avg_wu2', 'call_count_slope_wu2', 'call_count_avg_wu